In [1]:
import pandas as pd
import numpy as np
import flow

In [2]:
df = flow.get_population_data()
df.head()

,STATE FIPS,COUNTY FIPS,STATE/TERRITORY NAME,COUNTY NAME,2010 POPULATION,2015 POPULATION,LAND AREA (Square Miles)
0,1,1,Alabama,Autauga County,54571,55347.0,594.436
1,1,3,Alabama,Baldwin County,182265,203709.0,1589.784
2,1,5,Alabama,Barbour County,27457,26489.0,884.876
3,1,7,Alabama,Bibb County,22915,22583.0,622.582
4,1,9,Alabama,Blount County,57322,57673.0,644.776
